# Abstract

Test out setting up test for whole spreadsheet run.

# Environment

In [1]:
from astropy.table import Table
import numpy as np

In [2]:
from jwst.lib import set_telescope_pointing as stp

# Library

In [3]:
def row_to_pointing(r, siaf=None):
    """Convert a row of the spreadsheet to a Pointing
    
    Parameters
    ----------
    r: dict
        A row of pointing information
        
    siaf: SIAF
        SIAF info to use. If None, a non-transformative
        SIAF will be used.
        
    Returns
    -------
    Pointing, vinfo_expected
        A `Pointing` instance filled out.
        A `WCS` instance with the expected V1 information.
    """
    if siaf is None:
        siaf = stp.SIAF(v2_ref=0., v3_ref=0., v3yangle=0., vparity=1.)

    q = np.asarray([
        r['SA_ZATTEST1'],        
        r['SA_ZATTEST2'],        
        r['SA_ZATTEST3'],        
        r['SA_ZATTEST4'],        
    ])
    
    j2fgs_matrix = np.asarray([
        r['SA_ZRFGS2J11'], r['SA_ZRFGS2J21'], r['SA_ZRFGS2J31'],
        r['SA_ZRFGS2J12'], r['SA_ZRFGS2J22'], r['SA_ZRFGS2J32'],
        r['SA_ZRFGS2J13'], r['SA_ZRFGS2J23'], r['SA_ZRFGS2J33'],
    ])
    
    fsmcorr = np.asarray([
        r['SA_ZADUCMDX'], r['SA_ZADUCMDY']
    ])
    
    p = stp.Pointing(q=q, j2fgs_matrix=j2fgs_matrix, fsmcorr=fsmcorr)
    
    v = stp.WCSRef(ra=r['Vra'], dec=r['Vdec'], pa=r['V3PAatV1'])
    
    return p, v

In [4]:
def wcs_equality(left, right):
    """Assert equality of to WCSRef instances"""
    assert np.isclose(left.ra, right.ra), 'RAs are different {} != {}'.format(left.ra, right.ra)
    assert np.isclose(left.dec, right.dec), 'DECs are different {} != {}'.format(left.dec, right.dec)
    assert np.isclose(left.pa, right.pa), 'PAs are different {} != {}'.format(left.pa, right.pa)

# Main

In [5]:
t = Table.read('../data/acs_tlm_data_4_stsci_dms_jitter_file_mod_RA_DEC_PA.csv')

In [6]:
pointings = [
    row_to_pointing(r)
    for r in t
]

In [7]:
pointings[0]

(Pointing(q=array([-0.36915286,  0.33763282,  0.05758533,  0.86395264]), j2fgs_matrix=array([-1.00444000e-03,  3.38145836e-03,  9.99993778e-01,  9.99999496e-01,
        -3.90000000e-14,  1.00444575e-03,  3.39649146e-06,  9.99994283e-01,
        -3.38145665e-03]), fsmcorr=array([0., 0.]), obstime=None),
 WCSRef(ra=348.927867, dec=-38.749239, pa=50.176708))

In [8]:
len(pointings)

20711

In [9]:
siaf = stp.SIAF(v2_ref=0., v3_ref=0., v3yangle=0., vparity = 1.)

In [10]:
wcsinfo, vinfo = stp.calc_wcs(pointings[0][0], siaf)

In [11]:
vinfo

WCSRef(ra=348.92786699856623, dec=-38.749239233976155, pa=50.17670770067501)

In [12]:
pointings[0][1]

WCSRef(ra=348.927867, dec=-38.749239, pa=50.176708)

In [13]:
wcs_equality(vinfo, pointings[0][1])

In [14]:
for idx, pv in enumerate(pointings):
    p, v = pv
    wcsinfo, vinfo = stp.calc_wcs(p, siaf)
    try:
        wcs_equality(vinfo, v)
    except AssertionError as e:
        print('{}'.format(idx))
        print(e)
            